# Baseline_model.ipynb

### setting up features

In [1]:
import pandas as pd
import numpy as np
import scipy
pd.set_option('precision', 5)

In [2]:
# load cleaned dataset
path = '/home/emilyboeke/'
master_app = pd.read_csv(path + 'merged2_master_app.csv', low_memory=False)

In [3]:
# forgot to output excluding the auto-generated index column.... so now I'm dropping it. Can be optimized later
master_app = master_app.drop(master_app.columns[0], axis=1)

# drop NaNs. drops to 597954 obs
#tracid            598132
#nat               614012
#(other variables have 614204 observations)
#master_app = master_app.dropna()

# change dec to binary number, so we can do summary stats on it
master_app.loc[(master_app["dec"] == 'DENY'),'dec'] = 0
master_app.loc[(master_app["dec"] == 'GRANT'),'dec'] = 1

# delete NTA dates before 1984. nObs = 
master_app['osc_date'] = pd.to_datetime(master_app['osc_date'],infer_datetime_format = True)
master_app = master_app[master_app.osc_date.dt.year>1984]

#master_app.describe()

In [4]:
# selecting relevant features and target variable.
df = master_app[['osc_date', 'tracid', 'nat', 'dec']]

#print(df.head(10))
#df.describe()

In [5]:
# CHANGE OSC_DATE TO CONTINUOUS NUMBER.

osc_date_cont = []
startdate = np.datetime64('1984-01-01') # earliest date. from which timedelta is calculated

# change osc_date to continuous number
for i in df.index:
    x = df.loc[i,'osc_date'] - startdate
    osc_date_cont.append(x.days)
    
osc_date_cont = np.array(osc_date_cont)

In [6]:
# ONE HOT ENCODE CATEGORICAL VARIABLES

# change string nationalities to integer categories
from sklearn.preprocessing import LabelEncoder  
le = LabelEncoder()
nat_int = le.fit_transform(df['nat'])
nat_int = np.reshape(nat_int,[len(nat_int),1])

# get N x 2 array of features
feat_int = np.concatenate((df[['tracid']], nat_int), axis=1)
feat_int.shape

# get one hot encoder of features
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(feat_int)  
enc.feature_indices_

# create sparse matrix of all observations in Compressed Sparse Row format
blah = enc.transform(feat_int)

array([  0, 403, 631])

In [62]:
# CONCATENATING ONE HOT ENCODED FEATURES WITH CONTINUOUS FEATURES

# changes csr to csc, bc simpler to work with columns than rows
blah = scipy.sparse.csr_matrix.tocsc(blah)

# concatenating relevant fields
new_data = np.concatenate((blah.data, osc_date_cont)) # non-zero values in matrix
new_indices = np.concatenate((blah.indices, range(len(osc_date_cont)))) # row indices for each column
new_ind_ptr = np.append(blah.indptr, blah.indptr[-1]+len(osc_date_cont))

# making new matrix
from scipy.sparse import csc_matrix
final_feature_mat = csc_matrix((new_data, new_indices, new_ind_ptr))

### splitting into train, validation, and test data sets

In [18]:
# SAVE THE LAST 20% OF DATA AND DO NOT TOUCH IT!

# random seed

# shuffle data

idx = round(feat_int.shape[0] * .8)
X = feat_int[0:idx]
y = df['dec'][0:idx]

X_test_donttouch = feat_int[idx:-1]
y_test_donttouch = df['dec'][idx:-1]


### implementing logistic regression with ridge penalty

In [ ]:
# ridge regression


In [48]:
# ten-fold cross-validation